In [10]:
%run package_import.ipynb
%run data_profiling.ipynb

In [2]:
from sqlalchemy import create_engine
import pymysql

sqlEngine       = create_engine('mysql+pymysql://root:@127.0.0.1/kaggle_hour_price', pool_recycle=3600)
dbConnection    = sqlEngine.connect()

In [21]:
import os
os.chdir(path='/Users/liuyifu/Project_data')

In [105]:
df_combined = pd.read_sql_query(

''' 
select *
from raw_train
union all
select *, 
    cast(null as float) as SalePrice
from raw_test

'''



,dbConnection)

In [107]:
df_profile = main_data_profiling(df_combined)

In [108]:
df_profile

,column_names,data_type,unique_count,min_value,max_value,mean_value,median_value,null_values,total_rows,%_null_values,num_lower_outliers,num_higher_outliers
0,Id,int64,2919,1.0,2919.0,1460.000000,1460.0,0,2919,0.000000,0.0,0.0
1,MSSubClass,int64,16,20.0,190.0,57.137718,50.0,0,2919,0.000000,0.0,207.0
2,MSZoning,object,6,NaN,NaN,NaN,NaN,4,2919,0.001370,NaN,NaN
3,LotFrontage,float64,129,21.0,313.0,69.305795,68.0,486,2919,0.166495,104.0,94.0
4,LotArea,int64,1951,1300.0,215245.0,10168.114080,9453.0,0,2919,0.000000,1.0,127.0
5,Street,object,2,NaN,NaN,NaN,NaN,0,2919,0.000000,NaN,NaN
6,Alley,object,3,NaN,NaN,NaN,NaN,0,2919,0.000000,NaN,NaN
7,LotShape,object,4,NaN,NaN,NaN,NaN,0,2919,0.000000,NaN,NaN
8,LandContour,object,4,NaN,NaN,NaN,NaN,0,2919,0.000000,NaN,NaN
9,Utilities,object,3,NaN,NaN,NaN,NaN,2,2919,0.000685,NaN,NaN


### ETL Process

In [76]:
desc_dict, attr_dict = {}, {}
with open('House_Prices/data_description.txt', encoding='utf8') as f:
    for line in f:
        words = line.split()
        if words and ':' in line:
            col = words[0].replace(':','')
            desc_dict[col] = {}
        elif  not words:
            continue
        else:
            attr_desc = ' '.join(words[1:])
            attr = words[0]
            desc_dict[col][attr] = attr_desc

In [106]:
for col in df_combined.columns:
    if df_profile[df_profile['column_names'] == col]['null_values'].squeeze() > 0:
        if col in desc_dict and desc_dict[col].get('NA'):
            df_combined[col] = df_combined[col].fillna(desc_dict[col]['NA'])

#### 1. Replace NA

In [18]:
df_combined['Alley'] = df_combined['Alley'].fillna('No alley access')
df_combined['PoolQC'] = df_combined['PoolQC'].fillna('No Pool')
df_combined['PoolQC'] = df_combined['PoolQC'].fillna('No Pool')

#### 2. Remove NA